In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
236,AFG,Asia,Afghanistan,2020-08-23,37999.0,105.0,64.000,1387.0,2.0,2.429,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
404,ALB,Europe,Albania,2020-08-23,8275.0,156.0,145.000,245.0,5.0,2.857,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
641,DZA,Africa,Algeria,2020-08-23,41068.0,401.0,419.286,1424.0,6.0,9.143,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-22,France,237970.0,0.0,30535.0,0.0,0.0,0.0
0,2020-08-22,China,89660.0,0.0,4718.0,0.0,0.0,0.0
0,2020-08-22,Italy,257875.0,0.0,35454.0,0.0,0.0,0.0
0,2020-08-22,Spain,392530.0,0.0,28861.0,0.0,0.0,0.0
0,2020-08-22,United States,5708900.0,0.0,176566.0,0.0,0.0,0.0
0,2020-08-22,World,23432747.0,0.0,808066.0,0.0,0.0,0.0
0,2020-08-22,United Kingdom,324600.0,0.0,41448.0,0.0,0.0,0.0
0,2020-08-22,Germany,232097.0,0.0,9273.0,0.0,0.0,0.0
0,2020-08-22,Iran,358283.0,0.0,20648.0,0.0,0.0,0.0
0,2020-08-22,Turkey,256939.0,0.0,6097.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-23,France,237970.0,238002.0,30535.0,30512.0,-32.0,23.0
0,2020-08-23,China,89660.0,89654.0,4718.0,4711.0,6.0,7.0
0,2020-08-23,Italy,257875.0,258136.0,35454.0,35430.0,-261.0,24.0
0,2020-08-22,Spain,392530.0,386054.0,28861.0,28838.0,6476.0,23.0
0,2020-08-23,United States,5708900.0,5668105.0,176566.0,176362.0,40795.0,204.0
0,2020-08-23,World,23432747.0,23243621.0,808066.0,805740.0,189126.0,2326.0
0,2020-08-23,United Kingdom,324600.0,324601.0,41448.0,41423.0,-1.0,25.0
0,2020-08-23,Germany,232097.0,232864.0,9273.0,9269.0,-767.0,4.0
0,2020-08-23,Iran,358283.0,356792.0,20648.0,20502.0,1491.0,146.0
0,2020-08-23,Turkey,256939.0,257032.0,6097.0,6102.0,-93.0,-5.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")